# Actualizar Datos desde el Sistema AAPS-API

Los datos del sistema AAPS-API pueden cambiar entre sesiones.

El siguiente botón, nos permite actualizar los datos para trabajar siempre con los datos más recientes.
Si tienes tus credenciales del sistema AAPS-API (nombre de usuario + contraseña), ingrésalos abajo para acceder a los datos.

Si ya los ingresaste alguna vez mediante esta herramienta, es probable que tengas un token de acceso guardado en tu sistema y no tengas que ingresarlos nuevamente. 

In [21]:
import ipywidgets as widgets
import requests
import requests_cache
import os, json
from os.path import dirname as up

requests_cache.install_cache()

In [23]:
host = '200.87.123.68'
protocol = 'http'

username_widget = widgets.Text(
    description='Usuario:',
    placeholder='Usuario del sistema AAPS-API',
)
password_widget = widgets.Password(
    description='Contraseña:',
    placeholder='Contraseña del usuario'
)
update_button = widgets.Button(
    description='Actualizar los datos!',
    button_style='success',
    disabled=True,
    tooltip='Actualiza los datos locales desde el sistema AAPS-API!',
    icon='download',
    layout=widgets.Layout(width='300px')
)
show_password_button = widgets.ToggleButton(
    value = False,
    button_style='info',
    tooltip='Mostrar Contraseña',
    disabled=True,
    icon='eye',
    layout=widgets.Layout(width='40px')
)

help_html = widgets.HTML()
help_html.layout.width='500px'

def set_help_html(username, password, show_pass):
    user_label = 'Nombre de Usuario:' if username != '' else ''
    pass_label = 'Contraseña:' if password != '' else ''
    encoded_pass = password if show_pass else '\u2022' * len(password)
    help_html.value = f"<font color='gray'>{user_label}</font> {username} <font color='gray'>{pass_label}</font> {encoded_pass}"

def on_username_change(change):
    set_help_html(change['new'], password_widget.value, show_password_button.value)
    if change['new'] == '':
        update_button.disabled = True
    elif password_widget.value != '':
        update_button.disabled = False

def on_password_change(change):
    set_help_html(username_widget.value,change['new'], show_password_button.value)
    if change['new'] == '':
        show_password_button.disabled = True
        update_button.disabled = True
    else:
        show_password_button.disabled = False
        if username_widget.value != '':
            update_button.disabled = False

def on_show_pass_change(change):
    set_help_html(username_widget.value, password_widget.value, change['new'])
        
def on_update_button_click(b):
    token_json_path = os.path.join(up(os.getcwd()), 'auth', 'token.json')
    
    try:
        if not os.path.exists(token_json_path):
            username = username_widget.value
            password = password_widget.value
            r = requests.post(f'{protocol}://{host}/api-token-auth/', json={'username':username, 'password':password})

            if 'non_field_errors' in r.json().keys():
                help_html.value = "<font color='red'>Las credenciales proporcionadas no son válidas.</font> Verifica tus credenciales. Si el problema persiste, trata de ingresar a través de la <a href='https://aaps-data.appspot.com/admin/'>aplicación administrativa</a>. Si no puedes ingresar a través de esa página tampoco, contacta al administrador/administradora."
            else:
                os.makedirs(up(token_json_path))
                with open(token_json_path,'w') as f:
                    json.dump(r.json(),f)
        else:
            with open(token_json_path,'r') as f:
                auth_token = json.load(f)['token']    
            
            headers = {'Authorization': f'Token {auth_token}'}
            r = requests.get('https://aaps-data.appspot.com/api/epsas/', headers=headers)
            help_html.value = r.text + str(r.from_cache)
                
    except Exception as e:
        help_html.value = str(e)
        
        
username_widget.observe(on_username_change, names='value')
password_widget.observe(on_password_change, names='value')
show_password_button.observe(on_show_pass_change, names='value')
update_button.on_click(on_update_button_click)

widgets.HBox([widgets.VBox([username_widget, widgets.HBox([password_widget, show_password_button]), update_button]), help_html])

In [20]:
r = requests.get('https://aaps-data.appspot.com/api/epsas/')
r.text

'[{"code":"EPSAS","name":"Empresa Pública Social De Agua y Saneamiento","state":"LP","category":"A"},{"code":"MYEPSA","name":"Epsa de prueba My Epsa","state":"LP","category":"A"},{"code":"SAGUAPAC","name":"Cooperativa de Servicios Públicos Santa Cruz S.R.L.","state":"SC","category":"A"},{"code":"SEMAPA","name":"Servicio de Agua Potable y Alcantarillado de Cochabamba","state":"CO","category":"A"},{"code":"AAPOS","name":"Administración Autónoma para Obras Sanitarias","state":"PO","category":"B"},{"code":"BUSTILLO","name":"Entidad Prestadora de Servicios de Agua Potable y Alcantarillado Sanitario Bustillo Mancomunitaria Social","state":"PO","category":"B"},{"code":"CAPAG","name":"Cooperativa de Agua Potable y Alcantarillado Guayaramerín R.L.","state":"BE","category":"B"},{"code":"COATRI","name":"Cooperativa de Servicios de Agua Potable y Alcantarillado Trinidad Ltda.","state":"BE","category":"B"},{"code":"COOPAGUAS","name":"Cooperativa de Servicios Públicos 1ro de Mayo Ltda.","state":"SC"